In [1]:
import pandas as pd
import numpy as np

# !pip3 install transformers
import transformers
from transformers import pipeline
# !pip3 install tensorflow
import tensorflow
# !pip3 install torch torchvision
import torch

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transformers.__version__

'4.17.0'

In [3]:
df = pd.read_csv("cleaned_transcripts.csv", index_col = 0)
df.head()

,title,link,description,duration,podcast_id,transcript,cleaned_transcripts,cleaned_description
0,Changes To Texas Voting Rules Worry Older Vote...,https://www.npr.org/2022/03/02/1084043800/chan...,"Election officials said more than 15,000 mail-...",842.0,1084043800,"[<div aria-label=""Transcript"" class=""transcrip...","Hi. My name is Joey (ph), and I'm a bike mech...","Election officials said more than 15,000 mail-..."
1,"At The State Of The Union, Biden Wooed Moderat...",https://www.npr.org/2022/03/02/1083868631/at-t...,"In his speech, the president emphasized polici...",1141.0,1083868631,"[<div aria-label=""Transcript"" class=""transcrip...",Hey there. It's the NPR POLITICS PODCAST. I'...,"In his speech, the president emphasized polici..."
2,"""Awake Not Woke"": How Republicans Are Defining...",https://www.npr.org/2022/02/28/1083491401/-awa...,At last week's Conservative Political Action C...,828.0,1083491401,"[<div aria-label=""Transcript"" class=""transcrip...","(Speaking Catalan). And I'm her mother, Gina ...",At last week's Conservative Political Action C...
3,Ketanji Brown Jackson Is The First Black Woman...,https://www.npr.org/2022/02/25/1083180663/keta...,Jackson's experiences working as a public defe...,1571.0,1083180663,"[<div aria-label=""Transcript"" class=""transcrip...","Hi, this is Teresa (ph) on Jekyll Island, Ga....",Jackson's experiences working as a public defe...
4,US Responds To Russian Invasion Of Ukraine Wit...,https://www.npr.org/2022/02/24/1082922707/us-r...,In a speech at the White House Thursday aftern...,796.0,1082922707,"[<div aria-label=""Transcript"" class=""transcrip...",Hey there. It's the NPR POLITICS PODCAST. It...,In a speech at the White House Thursday aftern...


In [4]:
text = df['cleaned_transcripts'][0]
text

" Hi. My name is Joey (ph), and I'm a bike mechanic here in Washington, D.C. I'm currently working on one of my customer's bikes. This podcast was recorded at... 10 p.m. on March 2. Things may have changed by the time that you hear this, but hopefully this bike will be ready to go to not one, not two, but all three branches of government and a single bike ride. Enjoy the show. We are getting into prime biking season here in Washington, D.C. I know. The weather is beautiful. I kind of wish we could get outside right now, but we're going to talk about politics instead, I guess. Yeah. Just give me 20 minutes, Juana, and then you could be on the bike. I promise. OK. OK. We got it. Hey there. It's the NPR POLITICS PODCAST. I'm Miles Parks. I cover voting. And I'm Juana Summers. I cover politics and racial justice. We've also got Ashley Lopez here with us from member station KUT in Austin, Texas. Hi, Ashley. Hey there. Texas, where you are, held the first primary race in the country last nig

### Extractive Summarization

In [11]:
!pip install bert-extractive-summarizer

In [6]:
from summarizer import Summarizer

body = 'Text body that you want to summarize with BERT'
body2 = 'Something else you want to summarize with BERT'
model = Summarizer()
model(body)
model(body2)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


'Something else you want to summarize with BERT'

In [7]:
model = Summarizer()
result = model(text, min_length=60)
full = ''.join(result)
print(full)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


My name is Joey (ph), and I'm a bike mechanic here in Washington, D.C. I'm currently working on one of my customer's bikes. I kind of wish we could get outside right now, but we're going to talk about politics instead, I guess. It'll be incumbent Republican Greg Abbott against Democrat Beto O'Rourke. But it was also the first Election Day since sweeping election changes in the state that drastically change how a lot of people cast ballots. And, you know, it - depending on where you live, you're not likely to have been affected by these aspects of the changes to the law too much because it was only a couple of counties that took advantage of unlimited polling hours and drive-through voting, mostly Harris County during the pandemic in order to make it safer to vote. That is where some of the biggest changes are being felt by voters. Do I have that right or can you explain a little bit more how that worked? So vote by mail in Texas - which, you're right, is limited to people who are over 

## Abstractive Summarization

In [8]:
bart_cnn_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [10]:
char_size = 1000
final_summ = []
for i in range(len(full)//char_size):
    start_i = char_size * i
    end_i = min(char_size *(i + 1), len(text) - 1)
    sent_summ = bart_cnn_summarizer(full[start_i:end_i], max_length=130, min_length=30, do_sample=False)
    final_summ += [sent_summ[0]['summary_text']]
final_summ = ' '.join(final_summ)
print(final_summ)

This was the first Election Day since sweeping election changes in the state that drastically change how a lot of people cast ballots. Only a couple of counties took advantage of unlimited polling hours and drive-through voting. Voting rights advocates say there is quite obviously going to be discriminatory effects from creating new limits on this type of voting. There was also some concern about provisions in the law that give poll-watchers broader authority.


## Calculate ROUGE Score

In [11]:
#text
reference = df['cleaned_description'][0]
reference

"Election officials said more than 15,000 mail-in ballots weren't completed properly after the state imposed new voting rules governing Tuesday's primary races.Election workers received thousands of calls from voters with questions and some vulnerable Texans opted to vote in person for the first time in years to ensure their ballot would be counted.Similar rule changes have been imposed in more than a dozen states since the 2020 election."

In [6]:
import numpy as np
import pandas as pd
import re
import time
from bs4 import BeautifulSoup
import math

In [7]:
!pip install rouge/requirements.txt
!pip install rouge-score


ERROR: Invalid requirement: 'rouge/requirements.txt'
Hint: It looks like a path. File 'rouge/requirements.txt' does not exist.


In [8]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeLsum'], use_stemmer=True)
scores = scorer.score(final_summ, reference)

NameError: name 'final_summ' is not defined

In [23]:
scores

{'rouge1': Score(precision=0.2972972972972973, recall=0.2894736842105263, fmeasure=0.29333333333333333),
 'rouge2': Score(precision=0.0273972602739726, recall=0.02666666666666667, fmeasure=0.027027027027027025),
 'rougeLsum': Score(precision=0.14864864864864866, recall=0.14473684210526316, fmeasure=0.14666666666666667)}

In [ ]:
#how do we get rouge scores of all the summaries (is it an average)?

## Baseline on all Summaries

In [12]:
from summarizer import Summarizer
from rouge_score import rouge_scorer

In [13]:
bart_cnn_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
model = Summarizer()
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeLsum'], use_stemmer=True)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
df.isna().sum()

title                   0
link                    0
description             0
duration                0
podcast_id              0
transcript              0
cleaned_transcripts     0
cleaned_description    24
dtype: int64

In [57]:
pd.isnull(df.iloc[742]["cleaned_description"])
pd.isnull(float('nan'))

True

In [58]:
rouge1_scores = []
rouge2_scores = []
rogeLsum_scores = []
summaries = []

for i in range(742, len(df)):
    text = df["cleaned_transcripts"][i]
    reference = df['cleaned_description'][i]
    
    result = model(text, min_length=60)
    full = ''.join(result)
    
    char_size = 1000
    final_summ = []
    for j in range(len(full)//char_size):
        start_i = char_size * j
        end_i = min(char_size *(j + 1), len(text) - 1)
        sent_summ = bart_cnn_summarizer(full[start_i:end_i], max_length=130, min_length=30, do_sample=False)
        final_summ += [sent_summ[0]['summary_text']]
    final_summ = ' '.join(final_summ)
    
    summaries += [final_summ]
    
    print(i)

    if pd.isnull(reference):
        rouge1_scores += [float('nan')]
        rouge2_scores += [float('nan')]
        rogeLsum_scores += [float('nan')]
    else:
        scores = scorer.score(final_summ, reference)

        rouge1_scores += [scores['rouge1']]
        rouge2_scores += [scores['rouge2']]
        rogeLsum_scores += [scores['rougeLsum']]
    

742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991


## Average ROUGE Scores

In [151]:
all_summaries = pd.read_csv("all_summaries.csv", index_col = 0)
all_summaries.head()

,title,rouge 1,rouge 2,rouge L sum,summary
0,Changes To Texas Voting Rules Worry Older Vote...,"Score(precision=0.2972972972972973, recall=0.2...","Score(precision=0.0273972602739726, recall=0.0...","Score(precision=0.14864864864864866, recall=0....",This was the first Election Day since sweeping...
1,"At The State Of The Union, Biden Wooed Moderat...","Score(precision=0.3448275862068966, recall=0.1...","Score(precision=0.03508771929824561, recall=0....","Score(precision=0.1724137931034483, recall=0.0...",President Biden delivered his first State of t...
2,"""Awake Not Woke"": How Republicans Are Defining...","Score(precision=0.20754716981132076, recall=0....","Score(precision=0.019230769230769232, recall=0...","Score(precision=0.09433962264150944, recall=0....",Domenico Montanaro: CPAC is now the Trump show...
3,Ketanji Brown Jackson Is The First Black Woman...,"Score(precision=0.4791666666666667, recall=0.1...","Score(precision=0.10638297872340426, recall=0....","Score(precision=0.3125, recall=0.1271186440677...",Teresa: She's only the second generation in he...
4,US Responds To Russian Invasion Of Ukraine Wit...,"Score(precision=0.4444444444444444, recall=0.2...","Score(precision=0.045454545454545456, recall=0...","Score(precision=0.2, recall=0.0978260869565217...",Vladimir Putin ignored months of warnings from...


In [102]:
import regex as re

In [147]:
rouge1_f = []
rouge2_f = []
rougeLsum_f = []
for i in range(len(all_summaries)):
    string1_val = all_summaries["rouge 1"][i]
    if not pd.isnull(string1_val):
        text1_scores = re.findall(r'\d\.?\d*', string1_val)
        rouge1_f += [float(text1_scores[2])]
        
    string2_val = all_summaries["rouge 2"][i]
    if not pd.isnull(string2_val):
        text2_scores = re.findall(r'\d\.?\d*', string2_val)
        rouge2_f += [float(text2_scores[2])]
        
    stringL_val = all_summaries["rouge L sum"][i]
    if not pd.isnull(stringL_val):
        textL_scores = re.findall(r'\d\.?\d*', stringL_val)
        rougeLsum_f += [float(textL_scores[2])]

In [148]:
np.mean(rouge1_f)

0.21901209441248012

In [149]:
np.mean(rouge2_f)

0.04969295143810813

In [150]:
np.mean(rougeLsum_f)

0.13577837466236206